In [4]:
import os
import sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' # specify GPUs locally

# libraries
import time
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm as tqdm
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
import random

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import roc_auc_score
import albumentations

import monai
from monai.data import NiftiDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor

# from apex import amp # I cannot install apex in Kagggle notebook

device = torch.device('cuda')

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    
set_seed(0)

In [1]:
import json
from pathlib import Path

setting_json = open("SETTINGS.json", "r")
setting_json = json.load(setting_json)

In [3]:
DEBUG = True

kernel_type = 'monai3d_160_3ch_1e-5_20ep_aug'

image_size = 100
use_amp = False
data_dir = f'{setting_json["TRAIN_IMAGES"]}/'
num_workers = 8
init_lr = 1e-5
out_dim = 6
freeze_epo = 0
warmup_epo = 1
cosine_epo = 12
n_epochs = freeze_epo + warmup_epo + cosine_epo
setting_json

{'OUTPUT': './output',
 'INPUT': '../input/rsna-str-pulmonary-embolism-detection',
 'TRAIN_IMAGES': '../input/rsna-str-pulmonary-embolism-detection/train-jpegs',
 'EXTERNAL': '../input/external_table_data'}

In [ ]:
target_cols = [
        'rv_lv_ratio_gte_1', # exam level
        "central_pe",
        "leftsided_pe",
        "rightsided_pe",
        "acute_and_chronic_pe",
        "chronic_pe"
    ]

In [7]:
traindf = pd.read_pickle(f'{setting_json["EXTERNAL"]}/train_with_position.pkl')
traindf = traindf.sort_values(['StudyInstanceUID', 'z_pos'])
traindf = traindf[(traindf.negative_exam_for_pe == 0) & (traindf.indeterminate == 0)]

In [34]:
grouped = traindf[["StudyInstanceUID", "SeriesInstanceUID", 'rv_lv_ratio_gte_1', # exam level
        "central_pe",
        "leftsided_pe",
        "rightsided_pe",
        "acute_and_chronic_pe",
        "chronic_pe"]].drop_duplicates().reset_index(drop = True)

In [5]:
grouped = pd.merge(grouped, pd.read_csv(f"{setting_json["INPUT"]}/split.csv"), on = "StudyInstanceUID")

In [14]:
from glob import glob
from monai.transforms import LoadNifti, Randomizable, apply_transform
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor, RandAffine
from monai.utils import get_seed

class RSNADataset3D(torch.utils.data.Dataset, Randomizable):
    def __init__(self, csv, mode, transform=None):

        self.csv = csv.reset_index()
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return self.csv.shape[0]
    
    def randomize(self) -> None:
        MAX_SEED = np.iinfo(np.uint32).max + 1
        self._seed = self.R.randint(MAX_SEED, dtype="uint32")    

    def __getitem__(self, index):
        try:
            self.randomize()
            row = self.csv.iloc[index]
            #print(os.path.join(data_dir, row.StudyInstanceUID, row.SeriesInstanceUID))
            jpg_lst = sorted(glob(os.path.join(data_dir, row.StudyInstanceUID, row.SeriesInstanceUID, '*.png')))

            jpg_lst = jpg_lst[int(len(jpg_lst) * 0.25 ): int(len(jpg_lst) * 0.75)]

            #print(jpg_lst[len()])
            img_lst = [cv2.imread(jpg)[75:425,75:425,::-1] for jpg in jpg_lst] 
            img = np.stack([image.astype(np.float32) for image in img_lst], axis=2).transpose(3,0,1,2)

            if self.transform is not None:
                if isinstance(self.transform, Randomizable):
                    self.transform.set_random_state(seed=self._seed)
                img = apply_transform(self.transform, img)

            if self.mode == 'test':
                return img
            else:
                return img, torch.tensor(row[target_cols]).float()
        except:
            self.randomize()
            row = self.csv.iloc[0]
            #print(os.path.join(data_dir, row.StudyInstanceUID, row.SeriesInstanceUID))
            jpg_lst = sorted(glob(os.path.join(data_dir, row.StudyInstanceUID, row.SeriesInstanceUID, '*.png')))

            jpg_lst = jpg_lst[int(len(jpg_lst) * 0.25 ): int(len(jpg_lst) * 0.75)]

            #print(jpg_lst[len()])
            img_lst = [cv2.imread(jpg)[100:400,100:400,::-1] for jpg in jpg_lst] 
            img = np.stack([image.astype(np.float32) for image in img_lst], axis=2).transpose(3,0,1,2)

            if self.transform is not None:
                if isinstance(self.transform, Randomizable):
                    self.transform.set_random_state(seed=self._seed)
                img = apply_transform(self.transform, img)

            if self.mode == 'test':
                return img
            else:
                return img, torch.tensor(row[target_cols]).float()

In [15]:
train_transforms = Compose([ScaleIntensity(), 
                            Resize((image_size, image_size, image_size)), 
                            RandAffine( 
                                      prob=0.6,
                                      translate_range=(10, 10, 10),
                                      rotate_range=(0, 0, 0.2),
                                      scale_range=(0.2, 0.2, 0.2),
                                      padding_mode='border'),
                            ToTensor()])
val_transforms = Compose([ScaleIntensity(), Resize((image_size, image_size, image_size)), ToTensor()])

In [18]:
w = torch.Tensor([1.0686+ 0.3562, 0.8549, 0.2850, 0.2850, 0.4749, 0.4749])
bce = nn.BCEWithLogitsLoss(w.cuda())
def criterion(logits, target): 
    loss = bce(logits, target)
    return loss

In [44]:
def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(data)       
        loss = criterion(logits, target)

        if not use_amp:
            loss.backward()
        else:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()

        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss


def val_epoch(model, loader, is_ext=None, n_test=1, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)
            LOGITS.append(logits.detach().cpu())
            TARGETS.append(target.detach().cpu())

    val_loss = criterion(torch.cat(LOGITS).cuda(), torch.cat(TARGETS).cuda()).cpu().detach().numpy()
    PROBS = torch.sigmoid(torch.cat(LOGITS)).numpy().squeeze()    
    LOGITS = torch.cat(LOGITS).numpy()
    TARGETS = torch.cat(TARGETS).numpy()
    
    #for i in range(6):
    #    print(target_cols[i], "mean unweighted logloss: ", log_loss(TARGETS[:, i], np.repeat(grouped[grouped.fold == 1][target_cols[3]].values.mean(), len(grouped[grouped.fold == 1][target_cols[3]].values))), "unweighted logloss:", log_loss(TARGETS[:, i], PROBS[:, i]), "roc_auc:",  roc_auc_score(TARGETS[:, i], PROBS[:, i]))
    for i in range(6):
        try:
            print(target_cols[i], "unweighted logloss:", log_loss(TARGETS[:, i], PROBS[:, i]), "roc_auc:",  roc_auc_score(TARGETS[:, i], PROBS[:, i]))
        except:
            continue
    if get_output:
        return LOGITS, PROBS, TARGETS
    else:
        acc = (PROBS.round() == TARGETS).mean() * 100.
        auc = roc_auc_score(TARGETS, LOGITS)
        return float(val_loss), acc, auc

In [20]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [21]:
def run(fold):
    df_train = grouped[(grouped['fold'] != fold)]
    df_valid = grouped[(grouped['fold'] == fold)]

    dataset_train = RSNADataset3D(df_train, 'train', transform=train_transforms)
    dataset_valid = RSNADataset3D(df_valid, 'val', transform=val_transforms)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=8, sampler=RandomSampler(dataset_train), num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=8, num_workers=num_workers)

    model = monai.networks.nets.densenet.densenet121(spatial_dims=3, in_channels=3, out_channels=out_dim).to(device)

    val_loss_best = 1000
    model_file = f'{kernel_type}_6targets_{fold}.pth'

    optimizer = optim.Adam(model.parameters(), lr=init_lr)
    if use_amp:
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
#     if len(os.environ['CUDA_VISIBLE_DEVICES'].split(',')) > 1:
#         model = nn.DataParallel(model)         
        
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, cosine_epo)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

    print(len(dataset_train), len(dataset_valid))

    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)
        scheduler_warmup.step(epoch-1)

        train_loss = train_epoch(model, train_loader, optimizer)
        val_loss, acc, auc = val_epoch(model, valid_loader)
    
        content = time.ctime() + ' ' + f'Fold {fold}, Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}, auc: {(auc):.6f}'
        print(content)
        with open(f'log_{kernel_type}.txt', 'a') as appender:
            appender.write(content + '\n')             
            
        torch.save(model.state_dict(), model_file[:-4] + "_" + str(val_loss) + ".pth")
        #if val_loss < val_loss_best:
        #    print('val_loss_best ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_best, val_loss))
        #    torch.save(model.state_dict(), model_file[:-4] + "_" + str(val_loss) + ".pth")
        #    val_loss_best = val_loss

    torch.save(model.state_dict(), f'{kernel_type}_model_fold{fold}.pth')

In [178]:
run(fold=0)

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1769 442
Mon Oct 19 22:45:47 2020 Epoch: 1


100%|██████████| 56/56 [04:18<00:00,  4.62s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.6806042518685845 roc_auc: 0.6022291666666666
central_pe unweighted logloss: 0.5100659811995688 roc_auc: 0.6107839704618518
leftsided_pe unweighted logloss: 0.6114190454127023 roc_auc: 0.5224942456580874
rightsided_pe unweighted logloss: 0.5233248228805637 roc_auc: 0.5408682568482542
acute_and_chronic_pe unweighted logloss: 0.5205205506329084 roc_auc: 0.4656424814227269
chronic_pe unweighted logloss: 0.4348298086030451 roc_auc: 0.4953945596552653
Mon Oct 19 23:07:42 2020 Fold 0, Epoch 1, lr: 0.0000100, train loss: 0.39398, valid loss: 0.36381, acc: 79.9397, auc: 0.539569
Mon Oct 19 23:07:42 2020 Epoch: 2


100%|██████████| 56/56 [04:15<00:00,  4.57s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:942: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

...

rv_lv_ratio_gte_1 unweighted logloss: 0.5089091098913712 roc_auc: 0.8344583333333333
central_pe unweighted logloss: 0.43684472519497713 roc_auc: 0.7785912085813413
leftsided_pe unweighted logloss: 0.5386310818761048 roc_auc: 0.7125706214689266
rightsided_pe unweighted logloss: 0.3916593160463404 roc_auc: 0.5687038102402973
acute_and_chronic_pe unweighted logloss: 0.24302111553845768 roc_auc: 0.57293145194957
chronic_pe unweighted logloss: 0.34258504921561994 roc_auc: 0.5953137624562349
Tue Oct 20 02:45:07 2020 Fold 0, Epoch 11, lr: 0.0000146, train loss: 0.24699, valid loss: 0.27363, acc: 83.2579, auc: 0.677095
Tue Oct 20 02:45:07 2020 Epoch: 12


  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.4832089225491888 roc_auc: 0.8443124999999999
central_pe unweighted logloss: 0.40062560814609793 roc_auc: 0.8122354139478626
leftsided_pe unweighted logloss: 0.5262255715248407 roc_auc: 0.7101642603055032
rightsided_pe unweighted logloss: 0.38314950275888066 roc_auc: 0.6053015887064653
acute_and_chronic_pe unweighted logloss: 0.24354821642886054 roc_auc: 0.5591550471737496
chronic_pe unweighted logloss: 0.3422617382386792 roc_auc: 0.601992997576084
Tue Oct 20 03:06:43 2020 Fold 0, Epoch 12, lr: 0.0000067, train loss: 0.24095, valid loss: 0.26139, acc: 83.7858, auc: 0.688860
Tue Oct 20 03:06:43 2020 Epoch: 13


100%|██████████| 56/56 [04:15<00:00,  4.56s/it]


rv_lv_ratio_gte_1 unweighted logloss: 0.48821599758335504 roc_auc: 0.8415833333333333
central_pe unweighted logloss: 0.4023042164176146 roc_auc: 0.8089569341439348
leftsided_pe unweighted logloss: 0.5296060120037913 roc_auc: 0.7091441724210085
rightsided_pe unweighted logloss: 0.3843601661106378 roc_auc: 0.6020710713811567
acute_and_chronic_pe unweighted logloss: 0.24361813076396738 roc_auc: 0.5527260582783668
chronic_pe unweighted logloss: 0.34405514012971616 roc_auc: 0.6068408295179101
Tue Oct 20 03:28:29 2020 Fold 0, Epoch 13, lr: 0.0000017, train loss: 0.23825, valid loss: 0.26318, acc: 83.9744, auc: 0.686887


In [179]:
run(fold=1)

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1768 443
Tue Oct 20 03:28:30 2020 Epoch: 1


100%|██████████| 56/56 [04:16<00:00,  4.58s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/221 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.6805717870424887 roc_auc: 0.5937848210969419
central_pe unweighted logloss: 0.533625132276027 roc_auc: 0.5813639912000568
leftsided_pe unweighted logloss: 0.6115618692563835 roc_auc: 0.5396940097677664
rightsided_pe unweighted logloss: 0.5129201087268158 roc_auc: 0.5102642665645347
acute_and_chronic_pe unweighted logloss: 0.45027272998613915 roc_auc: 0.5784528656435953
chronic_pe unweighted logloss: 0.5021545215600232 roc_auc: 0.5484693877551019
Tue Oct 20 03:50:02 2020 Fold 1, Epoch 1, lr: 0.0000100, train loss: 0.40502, valid loss: 0.36644, acc: 79.3830, auc: 0.558672
Tue Oct 20 03:50:02 2020 Epoch: 2


100%|██████████| 56/56 [04:15<00:00,  4.57s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:942: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
  0%|          | 0/221 [00:00<?, ?it/s]

...

rv_lv_ratio_gte_1 unweighted logloss: 0.5781384986214567 roc_auc: 0.7892781973505015
central_pe unweighted logloss: 0.40820808295353944 roc_auc: 0.7758498332268824
leftsided_pe unweighted logloss: 0.5738826338895523 roc_auc: 0.6251619655138044
rightsided_pe unweighted logloss: 0.4067400898963095 roc_auc: 0.6751819226350058
acute_and_chronic_pe unweighted logloss: 0.2575055295370233 roc_auc: 0.5050109614782337
chronic_pe unweighted logloss: 0.337097610702263 roc_auc: 0.6710684273709484
Tue Oct 20 07:23:07 2020 Fold 1, Epoch 11, lr: 0.0000146, train loss: 0.23867, valid loss: 0.28909, acc: 82.0918, auc: 0.673592
Tue Oct 20 07:23:07 2020 Epoch: 12


  0%|          | 0/221 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.5771396220346353 roc_auc: 0.7898559696256862
central_pe unweighted logloss: 0.4123681690380961 roc_auc: 0.7746788730395288
leftsided_pe unweighted logloss: 0.5750730121140337 roc_auc: 0.6234426392903418
rightsided_pe unweighted logloss: 0.4094925764107206 roc_auc: 0.6582152432018383
acute_and_chronic_pe unweighted logloss: 0.25739944486145105 roc_auc: 0.530300657688694
chronic_pe unweighted logloss: 0.3365358260815488 roc_auc: 0.6729191676670668
Tue Oct 20 07:44:25 2020 Fold 1, Epoch 12, lr: 0.0000067, train loss: 0.23189, valid loss: 0.28958, acc: 82.1294, auc: 0.674902
Tue Oct 20 07:44:25 2020 Epoch: 13


100%|██████████| 56/56 [04:11<00:00,  4.49s/it]


rv_lv_ratio_gte_1 unweighted logloss: 0.5624111860824442 roc_auc: 0.7925797532086996
central_pe unweighted logloss: 0.41566016367120506 roc_auc: 0.7779433681073025
leftsided_pe unweighted logloss: 0.5728705471652743 roc_auc: 0.6210754510116615
rightsided_pe unweighted logloss: 0.40736782486901596 roc_auc: 0.669207200306396
acute_and_chronic_pe unweighted logloss: 0.2561467587800684 roc_auc: 0.524036955840902
chronic_pe unweighted logloss: 0.3326119419616178 roc_auc: 0.6803221288515406
Tue Oct 20 08:05:36 2020 Fold 1, Epoch 13, lr: 0.0000017, train loss: 0.22907, valid loss: 0.28594, acc: 82.3175, auc: 0.677527


In [ ]:
run(fold=2)

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1769 442
Tue Oct 20 08:05:38 2020 Epoch: 1


100%|██████████| 56/56 [04:06<00:00,  4.40s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.6598670919556423 roc_auc: 0.6635249366018596
central_pe unweighted logloss: 0.5414252523922812 roc_auc: 0.6646894051738342
leftsided_pe unweighted logloss: 0.6265587385962991 roc_auc: 0.4711098398169336
rightsided_pe unweighted logloss: 0.4950333655352506 roc_auc: 0.49768654524089306
acute_and_chronic_pe unweighted logloss: 0.3887430720739235 roc_auc: 0.5257026627218935
chronic_pe unweighted logloss: 0.5221017897129059 roc_auc: 0.5665312630499981
Tue Oct 20 08:26:52 2020 Fold 2, Epoch 1, lr: 0.0000100, train loss: 0.38948, valid loss: 0.35921, acc: 79.0724, auc: 0.564874
Tue Oct 20 08:26:52 2020 Epoch: 2


100%|██████████| 56/56 [04:04<00:00,  4.36s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:942: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

...

  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.5767444948122175 roc_auc: 0.7482248520710059
central_pe unweighted logloss: 0.43742810354496425 roc_auc: 0.7307628933926512
leftsided_pe unweighted logloss: 0.6114292024167011 roc_auc: 0.5872663996948895
rightsided_pe unweighted logloss: 0.4362023583807186 roc_auc: 0.6268727967097532
acute_and_chronic_pe unweighted logloss: 0.22334091852490717 roc_auc: 0.5673076923076923
chronic_pe unweighted logloss: 0.441019186640487 roc_auc: 0.5861204965642914
Tue Oct 20 10:13:23 2020 Fold 2, Epoch 6, lr: 0.0000750, train loss: 0.27898, valid loss: 0.30163, acc: 80.4676, auc: 0.641093
Tue Oct 20 10:13:23 2020 Epoch: 7


  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.5666598430294776 roc_auc: 0.7772612003381235
central_pe unweighted logloss: 0.4140056433894659 roc_auc: 0.7737683308617564
leftsided_pe unweighted logloss: 0.5895312601064092 roc_auc: 0.6614702517162472
rightsided_pe unweighted logloss: 0.4334394155571667 roc_auc: 0.6314262632197414
acute_and_chronic_pe unweighted logloss: 0.22353384841857182 roc_auc: 0.5568602071005917
chronic_pe unweighted logloss: 0.427286132782428 roc_auc: 0.5828556243119092
Tue Oct 20 10:35:00 2020 Fold 2, Epoch 7, lr: 0.0000629, train loss: 0.27450, valid loss: 0.29366, acc: 80.4299, auc: 0.663940
Tue Oct 20 10:35:00 2020 Epoch: 8


loss: 0.34419, smth: 0.26595:  65%|██████▍   | 144/222 [11:14<02:26,  1.88s/it]

In [ ]:
run(fold=3)

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1769 442
Tue Oct 20 12:29:00 2020 Epoch: 1


100%|██████████| 56/56 [04:16<00:00,  4.57s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.6736876961872049 roc_auc: 0.5792815371762741
central_pe unweighted logloss: 0.5310533021774767 roc_auc: 0.6654686741136474
leftsided_pe unweighted logloss: 0.6334402894946785 roc_auc: 0.5587737168945288
rightsided_pe unweighted logloss: 0.4996128991853058 roc_auc: 0.4745062604879308
acute_and_chronic_pe unweighted logloss: 0.43546410066779384 roc_auc: 0.554126213592233
chronic_pe unweighted logloss: 0.4950026358010003 roc_auc: 0.5686757215619694
Tue Oct 20 12:50:08 2020 Fold 3, Epoch 1, lr: 0.0000100, train loss: 0.39852, valid loss: 0.36311, acc: 79.0724, auc: 0.566805
Tue Oct 20 12:50:08 2020 Epoch: 2


100%|██████████| 56/56 [04:14<00:00,  4.55s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:942: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

...

Tue Oct 20 15:18:02 2020 Epoch: 9


  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.5795482734420518 roc_auc: 0.7806599832915622
central_pe unweighted logloss: 0.4017807419687863 roc_auc: 0.8304678646592197
leftsided_pe unweighted logloss: 0.5911337427322951 roc_auc: 0.6359686801993079
rightsided_pe unweighted logloss: 0.38156922192552983 roc_auc: 0.6568564175379716
acute_and_chronic_pe unweighted logloss: 0.24849803398923773 roc_auc: 0.5425566343042071
chronic_pe unweighted logloss: 0.38891231442687874 roc_auc: 0.621434634974533
Tue Oct 20 15:39:21 2020 Fold 3, Epoch 9, lr: 0.0000371, train loss: 0.25936, valid loss: 0.29152, acc: 81.3348, auc: 0.677991
Tue Oct 20 15:39:22 2020 Epoch: 10


loss: 0.22799, smth: 0.23951:  26%|██▌       | 57/222 [04:54<14:12,  5.17s/it] 


KeyboardInterrupt: 

In [ ]:
run(fold=4)

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1769 442
Tue Oct 20 15:49:03 2020 Epoch: 1


100%|██████████| 56/56 [04:19<00:00,  4.64s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.6629508653512368 roc_auc: 0.6284597574186971
central_pe unweighted logloss: 0.4810492088891802 roc_auc: 0.6388794567062818
leftsided_pe unweighted logloss: 0.6495623357290596 roc_auc: 0.5214284098875999
rightsided_pe unweighted logloss: 0.5054865071541583 roc_auc: 0.4790540540540541
acute_and_chronic_pe unweighted logloss: 0.4150394346918995 roc_auc: 0.4820906737914336
chronic_pe unweighted logloss: 0.49935450495907624 roc_auc: 0.6102716468590831
Tue Oct 20 16:10:43 2020 Fold 4, Epoch 1, lr: 0.0000100, train loss: 0.39941, valid loss: 0.35321, acc: 79.5249, auc: 0.560031
Tue Oct 20 16:10:43 2020 Epoch: 2


100%|██████████| 56/56 [04:20<00:00,  4.65s/it]
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:942: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
  0%|          | 0/222 [00:00<?, ?it/s]

...

  0%|          | 0/222 [00:00<?, ?it/s]

Tue Oct 20 17:15:38 2020 Epoch: 5


  0%|          | 0/222 [00:00<?, ?it/s]

rv_lv_ratio_gte_1 unweighted logloss: 0.658320599094421 roc_auc: 0.728466129957304
central_pe unweighted logloss: 0.4810366543911701 roc_auc: 0.7625212224108657
leftsided_pe unweighted logloss: 0.7045225577209803 roc_auc: 0.5956532555352014
rightsided_pe unweighted logloss: 0.42534642360211095 roc_auc: 0.6576576576576576
acute_and_chronic_pe unweighted logloss: 0.2435959867745368 roc_auc: 0.5105619103281289
chronic_pe unweighted logloss: 0.38501803730340567 roc_auc: 0.6635823429541597
Tue Oct 20 17:36:55 2020 Fold 4, Epoch 5, lr: 0.0000854, train loss: 0.29327, valid loss: 0.32829, acc: 78.0920, auc: 0.653074
Tue Oct 20 17:36:55 2020 Epoch: 6


 57%|█████▋    | 32/56 [02:23<00:34,  1.43s/it]